In [1]:
#Standard Packages 
import sys 
import pandas as pd 
import numpy as np 
import datetime 
import py_effo.oracle_connection as oc

import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels


#DataRobot packages
import os 
import sys

import datarobot as dr 
import ssl 
import warnings 
import requests 
import contextlib

from py_effo.oracle_connection import OracleConnection

MAX_WAIT = 1200 # Maximum number of seconds to wait for prediction job to finish

API_TOKEN = 'PSkEVQYkd4QoTHFaHfd3fmJA2fH25aox' 
USERNAME = 'a260157' 

#path5='./output/confusion_csv/outtime_sps_ensemble_tpr_produce_dept.csv'



stores=25
quant=0.25

model='SPS ENSEMBLE'
sub='Overall'
grp='25thPctl_Overall'
grp2='25thPctl_Overall'

path=(str('./output/confusion_output/'+ model + sub + grp2 +'_confusion.csv'))

########################################################################################

#Trained on brands with units > 0 All Brands
#1. DV: SPS - Store Filter - Best Model

###########################################################################################
PROJECT_ID = '5e349eee490ec0356231a78f' 
MODEL_ID = '5e34a652b6628423f977ba29'



########################################################################################
table = 'VAKPI2_MERGE_ALLVARS4'

##############################################

#Oracle Workspace

############################################# 
WS = 'AN_RT_WS130'

predicted='prediction' 
actrank='ACTUAL_RANK' 
nbucket = 10 
bucket='PRED_GROUP' 
predrank='PRED_RANK'

In [2]:
iri_data=pd.read_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v3/prospecting_v3_modeling/01_input/iri_data_2015.csv')

iri_data.dtypes

brand_summary=iri_data.groupby(['BRAND'])['total_us_sales'].sum()
brand_summary = brand_summary.reset_index()
brand_summary.columns = ['BRAND', 'TOTAL_US_SALES']

brand_summary

con = OracleConnection(user='AN_RT_WS130', instance='exa_uskrgprdh')
raw_train_data = con.query('SELECT * FROM AN_RT_WS130.VAKPI2_MERGE_ALLVARS4 where PREPERIOD_SPS >0')

DF=raw_train_data
DF=DF[DF['NUMSTORES']>stores]
DF=DF[DF['KROGER_BRAND'] == 0]
#DF=DF[DF['RECAP_DEPT'].isin(['NATURAL FOODS'])]

DF= pd.merge(DF,brand_summary[['BRAND','TOTAL_US_SALES']],how='left',left_on=['BRAND'],right_on=['BRAND'])


DF=DF[DF['TOTAL_US_SALES'] <= 100000000]

DF=DF.fillna(0)
DF=DF.reset_index()

DF



# valid_data= valid_data[valid_data['actual_rank'] <= valid_data['qcut']]

# actual_score=valid_data.groupby(['qcut']).agg({'BRAND':'count', 'actual_score': "mean",'TARGET_CAGR_SALES':"mean",'TARGET_SALES_DIFF': "mean",'TARGET_SALES': "mean"}).reset_index()

# actual_score.columns = ['qcut','Count','Avg Target Score','Avg Target SALES CAGR','Avg Target SALES Diff','Avg Target Sales']

# actual_score['Method']="Top25Pctl"

# actual_score
# actual_score.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v21/03_output/02_model_validation/top25_pctl.csv')

,index,PRIMARY_DEPT,RECAP_DEPT,BRAND,CATEGORY,BRAND_SALES,BRAND_UNITS,BRAND_VISITS,BRAND_SPU,BRAND_SPV,...,CHG_PERCUNITS_TOTPROMO,CHG_PERCSTOREWKS_TPR,CHG_PERCSTOREWKS_DISP,CHG_PERCSTOREWKS_FLYER,CHG_PERCSTOREWKS_PROMO,TARGET_CAGR_SALES,TARGET_CAGR_SPS,KROGER_BRAND,NUM_OF_DAYS_GROUP,TOTAL_US_SALES
0,0,GROCERY,GROCERY,1 2 3,137,474683.11,211137,196072,2.248223,2.420963,...,0.068623,0.059484,-0.002970,0.003070,0.058530,-0.015529,-0.016569,0,5 - Over A Year,21300218.39
1,1,DRUG/GM,HBC,100 GRAND,201,72912.76,100280,59159,0.727092,1.232488,...,0.041855,0.063605,-0.001217,-0.067739,0.028391,0.043013,0.035068,0,5 - Over A Year,11875715.88
2,2,DRUG/GM,HBC,100 GRAND,202,567070.20,535136,392095,1.059675,1.446257,...,0.175704,0.042080,0.072868,0.009177,0.120553,-0.346183,-0.312099,0,5 - Over A Year,11875715.88
3,3,DRUG/GM,HBC,1234,678,27529.45,1384,1331,19.891221,20.683283,...,0.038333,0.036716,0.000000,0.000000,0.036716,-0.285369,-0.200655,0,4 - 181-360 days,964504.97
4,4,GROCERY,NATURAL FOODS,1907,629,5501.89,2185,1330,2.518027,4.136759,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.132835,0.225872,0,5 - Over A Year,469653.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,8511,DRUG/GM,HBC,ZUM,187,6330.36,844,689,7.500427,9.187750,...,0.053836,0.006226,-0.018359,0.000000,-0.012133,0.127155,-0.201320,0,5 - Over A Year,5816154.41
6709,8512,DRUG/GM,HBC,ZUM,217,42443.77,6741,5528,6.296361,7.677961,...,-0.246015,-0.195421,-0.188101,0.000000,-0.341258,0.258233,0.179784,0,5 - Over A Year,5816154.41
6710,8513,DRUG/GM,HBC,ZUM,225,202672.00,40505,28146,5.003629,7.200739,...,-0.256602,-0.241486,-0.201216,0.000000,-0.337428,0.257085,-0.026400,0,5 - Over A Year,5816154.41
6711,8514,DRUG/GM,HBC,ZUM,296,61790.30,7517,6293,8.220074,9.818894,...,-0.330815,-0.174770,-0.244449,0.000000,-0.361583,0.240797,-0.022906,0,5 - Over A Year,5816154.41


In [3]:
cagr_sales=DF

cagr_sales['TARGET_SALES_DIFF'] = cagr_sales['TARGET_SALES'] - cagr_sales['PREPERIOD_SALES']
cagr_sales['std_cagr_yr3'] = (cagr_sales['TARGET_CAGR_SALES'] - cagr_sales['TARGET_CAGR_SALES'].mean())/cagr_sales['TARGET_CAGR_SALES'].std(ddof=0)
cagr_sales['std_pyr'] = (cagr_sales['PREPERIOD_SALES'] - cagr_sales['PREPERIOD_SALES'].mean())/cagr_sales['PREPERIOD_SPS'].std(ddof=0)

cagr_sales['std_fyr3'] = (cagr_sales['TARGET_SALES'] - cagr_sales['TARGET_SALES'].mean())/cagr_sales['TARGET_SALES'].std(ddof=0)
cagr_sales['std_fyr3diff'] = (cagr_sales['TARGET_SALES_DIFF'] - cagr_sales['TARGET_SALES_DIFF'].mean())/cagr_sales['TARGET_SALES_DIFF'].std(ddof=0)

#dat[comp_score]= np.where(dat[std_growth]==0,dat['std_fyr3'],(dat['std_fyr3']*0.70+dat[std_growth]*0.30))

def score(dat,val,growth,std_growth,comp_score,rank,decile,catrank):
    dat=dat.dropna(subset=[std_growth])
    dat[comp_score]= np.where(dat[std_growth]==0,dat['std_fyr3diff'],(dat['std_fyr3diff']*0.50+dat[std_growth]*0.50))
    dat.sort_values(comp_score,ascending=False,inplace=True)
    dat[rank] = range(1, len(dat) + 1)
    dat[decile] = pd.qcut(dat[comp_score], 5, labels=np.arange(5, 0, -1))
    dat[catrank] = dat.groupby(['CATEGORY'])[comp_score].rank(method='dense',ascending=False).astype(int)
    return dat

valid_data=score(cagr_sales,'fyr3','cagr_year_3','std_cagr_yr3','actual_score','actual_rank','actual_decile','actual_category_rank')

valid_data=valid_data.drop(['std_cagr_yr3', 'std_pyr','std_fyr3','std_fyr3diff'], axis=1)

valid_data['overall']='q1'


def q1(x):
    return x.quantile(quant)

f = {'actual_rank': [q1]}
valid_data0 =valid_data.agg(f).reset_index()
valid_data0.columns = ['overall','qcut']
valid_data0['qcut'] = round(valid_data0['qcut'])
valid_data0

valid_data = pd.merge(valid_data,valid_data0,how='left',on=['overall'])


def get_status(df):
    if df['actual_rank'] <= df['qcut']:
        return 1
    else:
        return 0

valid_data['actual_winner'] =valid_data.apply(get_status, axis = 1)


valid_data

,index,PRIMARY_DEPT,RECAP_DEPT,BRAND,CATEGORY,BRAND_SALES,BRAND_UNITS,BRAND_VISITS,BRAND_SPU,BRAND_SPV,...,NUM_OF_DAYS_GROUP,TOTAL_US_SALES,TARGET_SALES_DIFF,actual_score,actual_rank,actual_decile,actual_category_rank,overall,qcut,actual_winner
0,1924,GROCERY,NATURAL FOODS,DAVES KILLER BREAD,671,13781381.21,2797448,2555336,4.926412,5.393178,...,5 - Over A Year,91042518.22,44180268.68,20.321207,1,1,1,q1,1679.0,1
1,960,GROCERY,GROCERY,BODYARMOR,010,1469455.54,1144653,449851,1.283756,3.266538,...,5 - Over A Year,15577160.92,27098315.11,13.214965,2,1,1,q1,1679.0,1
2,3289,GROCERY,NATURAL FOODS,HALO TOP,696,1823256.74,439423,264886,4.149206,6.883175,...,5 - Over A Year,1547514.79,23254188.24,11.300724,3,1,1,q1,1679.0,1
3,2508,GROCERY,GROCERY,FAIRLIFE,060,5491706.50,1785194,1239213,3.076252,4.431608,...,5 - Over A Year,98283550.34,20187298.08,9.635496,4,1,1,q1,1679.0,1
4,573,GROCERY,NATURAL FOODS,BAI,355,3426594.63,1752399,737376,1.955374,4.647011,...,5 - Over A Year,93324596.77,14046036.01,6.876226,5,1,1,q1,1679.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,3177,MEAT,SEAFOOD,GREAT AMERICAN SEAFOOD,435,4317327.26,324109,264047,13.320603,16.350601,...,5 - Over A Year,96768421.23,-6755028.53,-3.153175,6709,5,11,q1,1679.0,0
6709,1024,GROCERY,NATURAL FOODS,BOULDER,672,5684831.75,1995401,1557703,2.848967,3.649497,...,5 - Over A Year,39167592.53,-7055465.13,-3.315932,6710,5,79,q1,1679.0,0
6710,1376,GROCERY,GROCERY,CATS PRIDE,213,3622015.07,583603,464706,6.206300,7.794208,...,5 - Over A Year,81479222.45,-6625298.58,-3.831992,6711,5,10,q1,1679.0,0
6711,629,GROCERY,GROCERY,BELVITA,053,8265144.15,2683112,1841409,3.080432,4.488489,...,5 - Over A Year,561967.05,-10082476.34,-4.642922,6712,5,30,q1,1679.0,0


In [4]:

DF=valid_data
def get_prediction(DF, API_TOKEN, PROJECT_ID, MODEL_ID, MAX_WAIT):

    """
    get predictions
    
    inputs:
    DF          -   a dataframe containing KPI data for new HHs
    API_TOKEN   -   API token for the prediction model
    PROJECT_ID  -   project ID for the prediction model
    MODEL_ID    -   model ID for the prediction model
    MAX_WAIT    -   Time in seconds after which project creation is considered unsuccessful
    
    output:
    pred_df     -   a dataframe containing HSHD_IDs and predictions
    
    """

    # disable proxy
    try:
        del os.environ['http_proxy']
    except KeyError:
        pass
    
    try:
        del os.environ['https_proxy']
    except KeyError:
        pass  
    

     
    try:
        from functools import partialmethodget
    except ImportError:
        # Python 2 fallback: https://gist.github.com/carymrobbins/8940382
        from functools import partial
     
        class partialmethod(partial):
            def __get__(self, instance, owner):
                if instance is None:
                    return self
     
                return partial(self.func, instance, *(self.args or ()), **(self.keywords or {}))
     
     
    @contextlib.contextmanager
    def no_ssl_verification():
        old_request = requests.Session.request
        requests.Session.request = partialmethod(old_request, verify=False)
     
        warnings.filterwarnings('ignore', 'Unverified HTTPS request')
        yield
        warnings.resetwarnings()
     
        requests.Session.request = old_request
     
     
    with no_ssl_verification():
        dr.Client(endpoint='https://datarobot.int.8451.com/api/v2', token=API_TOKEN)
        project = dr.Project.get(PROJECT_ID)
        model = dr.Model.get(PROJECT_ID, MODEL_ID)
     
        # Upload dataset to make predictions
        pred_dataset = project.upload_dataset(DF)
     
        # Make predictions on your data
        pred_job = model.request_predictions(pred_dataset.id)
        predictions = pred_job.get_result_when_complete(max_wait = MAX_WAIT)
        
        
        
    
    # data wrangling on predictions
    #predictions.prediction= predictions.prediction.map(lambda x: x.lstrip('X'))  # remove X from predictions
    #predictions = predictions.set_index('row_id')  # make row_id as an index
    #pred_df = DF[['BRAND_NAME']].join(predictions[['prediction']], how='inner')  # join HSHD and prediction
     
    return predictions

pred=get_prediction(DF, API_TOKEN, PROJECT_ID, MODEL_ID, MAX_WAIT)


#Creating Deciles and Prediction Matrix
valid_data= DF.merge(pred,left_index = True, right_index = True)

valid_data=valid_data.rename(index=str,columns={"prediction": "predicted_score"})



In [5]:
def score(dat,comp_score,rank,decile,catrank):
    dat.sort_values(comp_score,ascending=False,inplace=True)
    dat[rank] = range(1, len(dat) + 1)
    dat[decile] = pd.qcut(dat[comp_score], 5, labels=np.arange(5, 0, -1))
    dat[catrank] = dat.groupby(['CATEGORY'])[comp_score].rank(method='dense',ascending=False).astype(int)
    return dat

valid_data=score(valid_data,'predicted_score','pred_rank','pred_decile','pred_category_rank')


def get_status(df):
    if df['pred_rank'] <= df['qcut']:
        return 1
    else:
        return 0

valid_data['predicted_winner'] =valid_data.apply(get_status, axis = 1)

valid_data=valid_data.reset_index()

valid_data = valid_data.sort_values('predicted_score', ascending=False)

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-19 15:36:59.506013. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [6]:
valid_data=valid_data.drop(['row_id', 'index'], axis=1)

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-19 15:36:59.510702. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [7]:
#valid_data2.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v21/03_output/02_model_validation/out_of_time_composite.csv')

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-19 15:36:59.523064. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


1. Confusion Matrix:

In [8]:
########################################################
#Compute All 4 Classes, TP, FP, FN and TN
####################################################

ccb=pd.crosstab(valid_data['actual_winner'], valid_data['predicted_winner'])

ccb3=ccb[ccb.astype(bool)].unstack().reset_index()

ccb3.columns = ['Predicted','Actual','Value']

def flag_df(df):
    if (df['Predicted']==1) and  (df['Actual'] == 1):
        return 'TP'
    elif (df['Predicted'] ==0) and (df['Actual'] == 0):
        return 'TN'
    elif (df['Predicted'] ==1) and (df['Actual'] == 0):
        return 'FP'
    elif (df['Predicted'] ==0) and (df['Actual'] == 1):
        return 'FN'
    elif (df['Predicted'] >1):
        return 'none'

ccb3['Classification'] = ccb3.apply(flag_df, axis = 1)
ccb3['total']=ccb3['Value'].sum()
ccb3['percentage']=ccb3['Value']/ccb3['total']
ccb3

###############################################################
###Compute Specificity (TPR) and Sensitivity (FPR)  ##############################
#############################################################

def flag_df(df):
    if (df['Classification']=='TP') or (df['Classification'] == 'FN'):
        return 'TPFN'
    elif (df['Classification']=='TN') or (df['Classification'] == 'FP'):
        return 'TNFP'

ccb3['Classification2'] = ccb3.apply(flag_df, axis = 1)
ccb3_tot2=ccb3.groupby(['Classification2'])['Value'].sum()
ccb3_tot2=ccb3_tot2.reset_index()
ccb3_tot2.columns=['Classification2','total2']
ccb3=ccb3.merge(ccb3_tot2,left_on=['Classification2'],right_on=['Classification2'],how='inner')


def flag_df(df):
    if (df['Predicted']==1) and  (df['Actual'] == 1):
        return 'TPR'
    elif (df['Predicted'] ==0) and (df['Actual'] == 0):
        return 'TNR'
    elif (df['Predicted'] ==1) and (df['Actual'] == 0):
        return 'FPR'
    elif (df['Predicted'] ==0) and (df['Actual'] == 1):
        return 'FNR'
    elif (df['Predicted'] >1):
        return 'none'

ccb3['Classification2'] = ccb3.apply(flag_df, axis = 1)

ccb3['percentage2']=ccb3['Value']/ccb3['total2']

###############################################################
###Compute Precision Rate (TP/TP+FP) ##############################
#############################################################

def flag_df(df):
    if (df['Classification']=='TP') or (df['Classification'] == 'FP'):
        return 'TPFP'
    else:
        return 'TNFN'

ccb3['Classification3'] = ccb3.apply(flag_df, axis = 1)


ccb3_tot3=ccb3.groupby(['Classification3'])['Value'].sum()
ccb3_tot3=ccb3_tot3.reset_index()
ccb3_tot3.columns=['Classification3','total3']
ccb3=ccb3.merge(ccb3_tot3,left_on=['Classification3'],right_on=['Classification3'],how='inner')

ccb3

def flag_df(df):
    if (df['Predicted']==1) and  (df['Actual'] == 1):
        return 'Precision-Good'
    elif (df['Predicted'] ==0) and (df['Actual'] == 0):
        return 'Precision-Bad'
    elif (df['Predicted'] ==1) and (df['Actual'] == 0):
        return 'Failure-Bad'
    elif (df['Predicted'] ==0) and (df['Actual'] == 1):
        return 'Failure-Good'
    elif (df['Predicted'] >1):
        return 'none'

ccb3['Classification3'] = ccb3.apply(flag_df, axis = 1)


ccb3['percentage3']=ccb3['Value']/ccb3['total3']

###############################################################
###Compute Error and Accuracy Rate (TP+TN/All) ##############################
#############################################################

def flag_df(df):
    if (df['Classification']=='TP') or (df['Classification'] == 'TN'):
        return 'Overall Accuracy Rate'
    else:
        return 'Overall Error Rate'

ccb3['Classification4'] = ccb3.apply(flag_df, axis = 1)


ccb3_tot3=ccb3.groupby(['Classification4'])['Value'].sum()
ccb3_tot3=ccb3_tot3.reset_index()
ccb3_tot3.columns=['Classification4','total4']
ccb3=ccb3.merge(ccb3_tot3,left_on=['Classification4'],right_on=['Classification4'],how='inner')

ccb3





ccb3['percentage4']=ccb3['total4']/ccb3['total']
ccb3



# ###############################################################
# ###Splitting and Merging ##############################
# #############################################################
ccb=ccb3[['Classification', 'Value','total','percentage']]

ccb2=ccb3[['Classification2','Value','total2','percentage2']]

ccb2=ccb2.rename(index=str,columns={                    
                                                          'Classification2':'Classification',\
                                                          'total2':'total',\
                                                          'percentage2':'percentage'})

ccb=ccb.append(ccb2,ignore_index=True)


ccb2_2=ccb3[['Classification3','Value','total3','percentage3']]

ccb2_2=ccb2_2.rename(index=str,columns={                    
                                                          'Classification3':'Classification',\
                                                          'total3':'total',\
                                                          'percentage3':'percentage'})

ccb=ccb.append(ccb2_2,ignore_index=True)






ccb2_3=ccb3[['Classification4','total4','total','percentage4']]

ccb2_3=ccb2_3.drop_duplicates()
#ccb2_3=ccb2_3.reset_index()


ccb2_3=ccb2_3.rename(index=str,columns={                    
                                                          'Classification4':'Classification',\
                                                          'total':'total',\
                                                            'total4':'Value',\
                                                            'percentage4':'percentage'})
 

#ccb2_3

#ccb

ccb=ccb.append(ccb2_3,ignore_index=True)
ccb
#ccb

ccb.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v3/prospecting_v3_modeling/03_output/02_model_validation/composite_score/sameweeks/confusion_matrix_composite_score.csv')


/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-19 15:36:59.539513. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [71]:
ccb

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-13 12:23:26.155436. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


,Classification,Value,total,percentage
0,TN,4169,6713,0.621034
1,TP,814,6713,0.121257
2,FN,865,6713,0.128854
3,FP,865,6713,0.128854
4,TNR,4169,5034,0.828168
5,TPR,814,1679,0.484812
6,FNR,865,1679,0.515188
7,FPR,865,5034,0.171832
8,Precision-Bad,4169,5034,0.828168
9,Precision-Good,814,1679,0.484812


Correlation Results:

In [9]:
valid_data2=valid_data[['predicted_score',\
                              'pred_rank',\
                              'pred_category_rank',\
                              'pred_decile',\
                              'actual_score',\
                              'actual_rank',\
                               'actual_decile',\
                               'actual_category_rank'
                             ]]

corr_output=valid_data2.corr(method='pearson').iloc[0:3,3:7].reset_index()
corr_output=corr_output.reset_index()
corr_output.columns=['Sno','variable','predicted_score','pred_rank','pred_category_rank']
corr_output
#corr_output.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v3/prospecting_v3_modeling/03_output/02_model_validation/composite_score/sameweeks/correlation_composite_score.csv')

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-19 16:00:45.988017. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


,Sno,variable,predicted_score,pred_rank,pred_category_rank
0,0,predicted_score,0.497946,-0.449627,-0.175031
1,1,pred_rank,-0.423925,0.500888,0.202620
2,2,pred_category_rank,-0.136835,0.155862,0.780898


Quintile Cross-Tab:

In [68]:
quint=pd.crosstab(valid_data['pred_decile'], valid_data['actual_decile'],normalize='index')

quint=quint[quint.astype(bool)].unstack().reset_index()



quint.columns = ['Predicted','Actual','Value']

quint
#quint.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v3/prospecting_v3_modeling/03_output/02_model_validation/composite_score/sameweeks/samestore_decile_composite_score.csv')

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-13 12:14:22.581657. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


,Predicted,Actual,Value
0,5,5,0.571109
1,5,4,0.204173
2,5,3,0.101266
3,5,2,0.057377
4,5,1,0.066270
5,4,5,0.219657
6,4,4,0.271982
7,4,3,0.221147
8,4,2,0.169151
9,4,1,0.117647


Actual Score by Decile:

In [69]:
actual_score=valid_data.groupby(['pred_decile']).agg({'BRAND':'count', 'actual_score': "mean",'TARGET_CAGR_SALES':"mean",'TARGET_SALES_DIFF': "mean",'TARGET_SALES': "mean"}).reset_index()

actual_score.columns = ['Quintile','Count','Avg Target Score','Avg Target SALES CAGR','Avg Target SALES Diff','Avg Target Sales']

actual_score

#actual_score.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v3/prospecting_v3_modeling/03_output/02_model_validation/composite_score/sameweeks/target_samestore_actual_decile_composite_score.csv')

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-13 12:14:22.600703. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


,Quintile,Count,Avg Target Score,Avg Target SALES CAGR,Avg Target SALES Diff,Avg Target Sales
0,5,1343,-0.525131,-0.564443,-215906.124974,3.571199e+05
1,4,1342,-0.113997,-0.236679,-56123.695842,4.476507e+05
2,3,1343,0.030136,-0.105883,-36874.524058,4.743576e+05
3,2,1342,0.130465,-0.030269,12230.140544,6.056375e+05
4,1,1343,0.478540,0.116094,450908.982353,1.745392e+06


Profile Mis-Classified Brands:

In [70]:
# def flag_df(df):
#     if (df['predicted_winner']==1) and  (df['actual_winner'] == 1):
#         return 'TP'
#     elif (df['predicted_winner'] ==0) and (df['actual_winner'] == 0):
#         return 'TN'
#     elif (df['predicted_winner'] ==1) and (df['actual_winner'] == 0):
#         return 'FP'
#     elif (df['predicted_winner'] ==0) and (df['actual_winner'] == 1):
#         return 'FN'
#     elif (df['predicted_winner'] >1):
#         return 'none'

# valid_data['Classification'] = valid_data.apply(flag_df, axis = 1)

# summary_class=valid_data.groupby(['Classification']).agg({'BRAND':'count', 'actual_score': "mean",'predicted_score': "mean",'TARGET_CAGR_SALES':"mean",'TARGET_SALES_DIFF': "mean",'TARGET_SALES': "mean", 'PREPERIOD_SALES':"mean",'NUMSTORES': "mean"}).reset_index()

# summary_class.columns = ['Classification','Count','Avg Target Score','Avg Pred Score','Avg Target SALES CAGR','Avg Target SALES Diff','Avg Target Sales', 'Avg PREPERIOD Sales','Avg Stores']

# summary_class

#summary_class.to_csv('/nfs/analysis/users/a260157/brand_accelerator/prospecting_v21/03_output/02_model_validation/composite_score/target_by_decile_profiling_variables_composite_score.csv')

/opt/anaconda/2.7/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-02-13 12:14:22.601882. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
